In [2]:
import pandas as pd 
import numpy as np
import random
import surprise as sp
import scipy
from surprise import Dataset, Reader, SVD, evaluate
from sklearn import cross_validation as cv
from scipy import linalg


ratingscol = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings = pd.read_csv('./ratings100k.dat', sep='::', names = ratingscol, engine='python')
ratings = ratings.drop('Timestamp', axis=1)

def sample(ratings, n, m):
    """
    Return a smaller matrix with top n users and top m items only
    @param ratings the ratings dataset 
    @param n number of users with most ratings
    @param m number of movies with most ratings
    @returns NxM matrix of USERxITEM ratings
    """

    n_users = ratings['UserID'].nunique()
    n_items = ratings['MovieID'].nunique()

    user_sample = ratings['UserID'].value_counts().head(n).index
    movie_sample = ratings['MovieID'].value_counts().head(m).index

    subset = ratings.loc[ratings['UserID'].isin(user_sample)].loc[ratings['MovieID'].isin(movie_sample)]
    return subset

/Users/priyamedberry/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [62]:
# %load 'funcs.py'
import surprise as sp
from surprise import AlgoBase
import pandas as pd
import numpy as np
import scipy
from sklearn import cross_validation as cv
import funcs as F
import itertools as it
from scipy import linalg

def sample(ratings, user_counts, movie_counts, n, m):
    """
    Return a smaller matrix with top n users and top m items only
    @param ratings the ratings dataset 
    @param user_counts Count values of user ratings
    @param movie_counts Movie count values
    @param n number of users with most ratings
    @param m number of movies with most ratings
    @returns NxM matrix of USERxITEM ratings
    """
    n_users = ratings['UserId'].nunique()
    n_items = ratings['MovieId'].nunique()

    user_sample = user_counts.head(n).index
    movie_sample = movie_counts.head(m).index

    print len(user_sample)
    print len(movie_sample)

    subset = ratings.loc[ratings['UserId'].isin(user_sample)].loc[ratings['MovieId'].isin(movie_sample)]
    # we don't need the timestamp
    del subset['Timestamp']
    return subset

def normalize_user_means(ratings):
    """
    @param Ratings pandas dataframe
    @returns user mean normalized dataframe
    """
    u_i_df = F.build_user_item_matrix(ratings)
    means = u_i_df.mean(axis=1)

    def f(row):
        u_id = row["UserId"]
        new_r = row["Rating"] - means[u_id]
        return new_r

    ratings['Rating'] = ratings.apply(f, axis = 1)
    return ratings


ratingscol = ['UserId', 'MovieId', 'Rating', 'Timestamp']
ratings = pd.read_csv('./ratings100k.dat', sep='::', names = ratingscol, engine='python')
ratings = ratings.drop('Timestamp', axis=1)
ratings = normalize_user_means(ratings)
ratings_matrix = F.build_user_item_matrix(ratings)
ratings_matrix = ratings_matrix.values

       UserId  MovieId  Rating   Timestamp
0           1      122     5.0   838985046
1           1      185     5.0   838983525
2           1      231     5.0   838983392
3           1      292     5.0   838983421
4           1      316     5.0   838983392
5           1      329     5.0   838983392
6           1      355     5.0   838984474
7           1      356     5.0   838983653
8           1      362     5.0   838984885
9           1      364     5.0   838983707
10          1      370     5.0   838984596
11          1      377     5.0   838983834
12          1      420     5.0   838983834
13          1      466     5.0   838984679
14          1      480     5.0   838983653
15          1      520     5.0   838984679
16          1      539     5.0   838984068
17          1      586     5.0   838984068
18          1      588     5.0   838983339
19          1      589     5.0   838983778
20          1      594     5.0   838984679
21          1      616     5.0   838984941
22         

In [4]:
def svd(matrix):
    U, s, Vh = scipy.linalg.svd(matrix)
    return U.shape,  s.shape, Vh.shape
    
svd(ratings_matrix)

((730, 730), (730,), (6373, 6373))

In [127]:
class MF():
    
    def __init__(self, R, k, lr, reg, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent factors
        - lr (float) : learning rate
        - reg (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.k = k
        self.lr = lr
        self.reg = reg
        self.iterations = iterations
        
    def train(self):
        """
        Gets the predicted rating of user i and item j for train set
        """
        # Initialize user and item latent feature matrice
        # P is a k x n matrix(n = number of users)
        # Q is a m x k matrix(m = number of items)
        self.P = np.random.normal(scale=1./self.k, size=(self.num_users, self.k))
        self.Q = np.random.normal(scale=1./self.k, size=(self.num_items, self.k))

        # Initialize the user/item biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)

        # Don't include ratings of 0 into mean because those items are unrated
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
            ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mae = self.mae()
            training_process.append((i, mae))
            if (i+1) % 1 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def sgd(self):
        """
        Perform stochastic gradient descent
        """
        users = []
        items = []
        predicted_ratings = []
        for i, j, r in self.samples:
            # Compute prediction and error
            # i = user index, j = item index, r = preexisting rating

            prediction = self.get_rating(i, j)
            err = (r - prediction)
            
            users.append(i)
            items.append(j)
            predicted_ratings.append(prediction)

            # Update biases
            self.b_u[i] += self.lr * (err - self.reg * self.b_u[i])
            self.b_i[j] += self.lr * (err - self.reg * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.lr * (err * self.Q[j, :] - self.reg * self.P[i,:])
            self.Q[j, :] += self.lr * (err * self.P[i, :] - self.reg * self.Q[j,:])
            
        data = ({'UserId': users, 'MovieId': items, 'Ratings': predicted_ratings})
        df = pd.DataFrame(data)
        return df

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def test(self, new_data):
        """
        Gets the predicted rating of user i and item j for test set
        """
        num_users, num_items = new_data.shape
        
        testset = [
            (i, j)
            for i in range(num_users)
            for j in range(num_items)
        ]
        
        users = []
        items = []
        predicted_ratings = []
        
        for i, j in testset:
            prediction = self.get_rating(i, j)
            users.append(i)
            items.append(j)
            predicted_ratings.append(prediction)
        
        data = ({'UserId': users, 'MovieId': items, 'Ratings': predicted_ratings})
        df = pd.DataFrame(data)
        print df
        return df
        
    
    def mae(self):
        """
        A function to compute the total mean absolute error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        error = error/np.count_nonzero(self.R)
        return np.sqrt(error)

    def full_matrix(self):
        """
        Compute the full matrix using the resultant biases, P and Q
        """
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [128]:
mf = MF(R=ratings_matrix, k=20, lr=0.005, reg=0.02, iterations=5)
mf.train()
mf.test(new_data = ratings_matrix)



Iteration: 1 ; error = 0.9238
Iteration: 2 ; error = 0.8963
Iteration: 3 ; error = 0.8821
Iteration: 4 ; error = 0.8729
Iteration: 5 ; error = 0.8656
         MovieId   Ratings  UserId
0              0  4.600249       0
1              1  3.853355       0
2              2  3.901731       0
3              3  3.802350       0
4              4  3.812059       0
5              5  4.504077       0
6              6  3.973608       0
7              7  4.004313       0
8              8  3.900667       0
9              9  4.055025       0
10            10  4.316318       0
11            11  3.832859       0
12            12  3.996679       0
13            13  4.251689       0
14            14  3.914732       0
15            15  4.424587       0
16            16  4.580215       0
17            17  4.031937       0
18            18  3.214265       0
19            19  3.877511       0
20            20  4.186461       0
21            21  3.977415       0
22            22  3.927628       0
23        

,MovieId,Ratings,UserId
0,0,4.600249,0
1,1,3.853355,0
2,2,3.901731,0
3,3,3.802350,0
4,4,3.812059,0
5,5,4.504077,0
6,6,3.973608,0
7,7,4.004313,0
8,8,3.900667,0
9,9,4.055025,0


In [ ]:
def train_matrix(ratings, factor):
    """
    Train a model and return it. Then we can use the model and evaluate it elsewhere
    @param ratings dataframe pandas dataframe to train on, with columns UserId, MovieId, Ratings
    @param n_folds number of folds for cross validation
    @returns List of (algo, test data)
    We can call methods such as `test` and `evaluate` on this object 
    """

    train_data, test_data = cv.train_test_split(ratings, test_size = 0.20)
    reader = sp.Reader(rating_scale=(1, 5))

    trainset = sp.Dataset.load_from_df(train_data, reader)
    testset = sp.Dataset.load_from_df(test_data, reader)
    trainset.split(n_folds = 5)

    algo = sp.SVD(n_factors = factor)

    for trainset, _ in trainset.folds():
        algo.train(trainset)
        
    testset = testset.build_full_trainset().build_testset()
    return (algo, testset)


In [13]:
def group_predictions_by_user(predictions):
    """
    @param List of Surprise predictions objects
    @returns Dict {uid: [P1, P2, ...PN]} hash mapping user id to top n predictions
    """
    p = sorted(predictions, key = lambda x: x.uid)

    groups = {}
    for k, g in it.groupby(p, lambda x: x.uid):
        groups[k] = sorted(list(g), key = lambda x: x.est, reverse = True)

    return groups


# For every item that a user would be rated, in top k
def calculate_catalog_coverage(ratings, predictions, k):
    """
    Calculate the catalog coverage of a model over a dataset
    @param ratings pandas dataframe with UserId, MovieId, Ratings. Must be the same set the model was trained on
    @param List Surprise predictions
    @oaram k Int the top k recommendations size
    @returns Float percentage of items recommended to at least one user
    """
    n_movies = ratings['MovieId'].nunique()

    movies_reccommended = set() # keep track of which movies are recommended. Note we only care about the number

    recommendations = group_predictions_by_user(predictions)
    for u_id, recs in recommendations.iteritems():
        movies_reccommended.update(map(lambda x: x.iid, recs[0:3]))

    return len(movies_reccommended) / float(n_movies)

In [14]:
def evaluate(algo, ratings, testset, top_k):
    """
    @param algo Surprise algorithm the model that was trained
    @oaram ratings The ratings it was trained on, in pandas Dataframe form (so we can calculate coverage)
    @param testset Surprise testset object, the data held out during cross-validation
    @returns Nested Dictionary {test: {rmse, mae}, train: {rmse, mae, cc}}
    We can use these to build up arrays for plotting.
    """

    ret = {}
    ret['test'] = {}
    ret['train'] = {}

    test_predictions = algo.test(testset)
    # see how it would do on the trainset to compare, comes with the algo object
    trainset = algo.trainset.build_testset()
    train_predictions = algo.test(trainset)

    # sticking evaluate in everything for grep, training is verbose
    ret['test']['rmse'] = sp.accuracy.rmse(test_predictions)
    ret['train']['rmse'] = sp.accuracy.rmse(train_predictions)

    ret['test']['mae'] = sp.accuracy.mae(test_predictions)
    ret['train']['mae'] = sp.accuracy.mae(train_predictions)

    # Hackish, baseline does not have a sense of "neighbors"
    if (algo.__module__ == "surprise.prediction_algorithms.knns"):
        ret['test']['cc'] = calculate_catalog_coverage(ratings, test_predictions, top_k)

    return ret

In [15]:
# run models with some different parameters and sizes
# samples = [ [1000, 10], [5000, 50], [100000, 1000], [5000000, 2000] ]
samples = [ [1000, 10], [5000, 50], [10000,100], [20000,200] ]
factors = [5, 10, 20, 40, 50, 75]

for _sample in samples:
    i, j = _sample
    _dataset = sample(ratings, i, j)
    for f in factors:
        MF, MF_test = train_matrix(_dataset, f)

        print "Evaluating MF with f of {}".format(f)
        evaluate(MF, _dataset, MF_test, 5)

Evaluating MF with f of 5
RMSE: 0.8913
RMSE: 0.7613
MAE:  0.6980
MAE:  0.5935
Evaluating MF with f of 10
RMSE: 0.8181
RMSE: 0.7762
MAE:  0.6593
MAE:  0.6065
Evaluating MF with f of 20
RMSE: 0.8875
RMSE: 0.7449
MAE:  0.6809
MAE:  0.5851
Evaluating MF with f of 40
RMSE: 0.8224
RMSE: 0.7292
MAE:  0.6640
MAE:  0.5719
Evaluating MF with f of 50
RMSE: 0.8981
RMSE: 0.7027
MAE:  0.7031
MAE:  0.5516
Evaluating MF with f of 75
RMSE: 0.9062
RMSE: 0.6469
MAE:  0.6933
MAE:  0.5098
Evaluating MF with f of 5
RMSE: 0.8463
RMSE: 0.7787
MAE:  0.6609
MAE:  0.6074
Evaluating MF with f of 10
RMSE: 0.8495
RMSE: 0.7635
MAE:  0.6596
MAE:  0.5983
Evaluating MF with f of 20
RMSE: 0.8384
RMSE: 0.7609
MAE:  0.6531
MAE:  0.5942
Evaluating MF with f of 40
RMSE: 0.8476
RMSE: 0.7268
MAE:  0.6625
MAE:  0.5658
Evaluating MF with f of 50
RMSE: 0.8664
RMSE: 0.7033
MAE:  0.6809
MAE:  0.5475
Evaluating MF with f of 75
RMSE: 0.8519
RMSE: 0.6598
MAE:  0.6638
MAE:  0.5152
Evaluating MF with f of 5
RMSE: 0.8515
RMSE: 0.7945
MA